In [ ]:
import pandas
import numpy
import scipy
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi

In [ ]:
data = pandas.read_excel('all_data_20_may.xls')

## Описание данных

In [ ]:
data.describe()

## Определяем типы данных

In [ ]:
data['ScoreObyazMA'] = pandas.to_numeric(data['ScoreObyazMA'])
data['ScoreObyazRU'] = pandas.to_numeric(data['ScoreObyazRU'])
data['ScoreObyazLIT'] = pandas.to_numeric(data['ScoreObyazLIT'])
data['ClassNumber'] = pandas.to_numeric(data['ClassNumber'])
data['SchoolTypeID'] = pandas.to_numeric(data['SchoolTypeID'])
data['SchoolKindID'] = pandas.to_numeric(data['SchoolKindID'])
data['TeacherAge'] = pandas.to_numeric(data['TeacherAge'])
data['BooksMaID'] = pandas.to_numeric(data['BooksMaID'])
data['BooksRuID'] = pandas.to_numeric(data['BooksRuID'])
data['BooksLiID'] = pandas.to_numeric(data['BooksLiID'])
data['LessonsMa'] = pandas.to_numeric(data['LessonsMa'])
data['LessonsRu'] = pandas.to_numeric(data['LessonsRu'])
data['LessonsLi'] = pandas.to_numeric(data['LessonsLi'])

## Пример выделения подмассива по классу

In [291]:
filter_class = pandas.DataFrame(data[(data['ClassNumber'] == 3) & (data['ScoreObyazMA'] == data['ScoreObyazMA'])],columns=['BooksMaID','ScoreObyazMA'])

## Смотрим распределение полученных оценок (min, max) и делим выборку на заданное количество частей

In [292]:
describe = filter_class['ScoreObyazMA'].describe()
print(describe)
#max
print("min ", describe['min'])
#min
print("max ", describe['max'])
divider = round((describe['max'] - describe['min'])/2, 1)
print("divider ", divider)

count    19466.000000
mean        11.459776
std          2.639316
min          0.000000
25%         10.000000
50%         12.000000
75%         13.000000
max         15.000000
Name: ScoreObyazMA, dtype: float64
min  0.0
max  15.0
divider  7.5


## Формируем массив, разделяющий выборку на заданное количество равных интервалов

In [293]:
k = 2
#общее количество записей по каждой книге
all_groups = filter_class.groupby('BooksMaID').size()
print(all_groups)
# ключи книг
#numpy.array(all_groups.index)
# в dataframe по строкам обозначим учебники, а по столбцам интервалы
df2 = pandas.DataFrame(columns=range(k), index=numpy.array(all_groups.index))
interval = range(k)
#print(df2)
#идем по учебникам
for i, row in df2.iterrows():
    first_divider,  second_divider = describe['min'], describe['min'] + divider
    #идем по интервалам
    for t in interval:
        if t == interval[len(interval)-1]:
            second_divider +=1
        #фильтруем оценки, которые укладываются в интервал
        count = filter_class[(filter_class['BooksMaID'] == i) & (filter_class['ScoreObyazMA'] >= first_divider) & (filter_class['ScoreObyazMA'] < second_divider)]
        df2.set_value(i, t, len(count['BooksMaID']))
        first_divider,  second_divider = first_divider + divider, second_divider + divider
print("таблица сопряженности")
df2['sum_rec'] = all_groups
df2.loc['sum_rows'] = 0
list1 = [df2[i].sum() for i in interval]
for i in interval:
    df2.set_value('sum_rows', i, list1[i])
df2.set_value('sum_rows', 'sum_rec', df2['sum_rec'].sum())
print(df2)

BooksMaID
2       65
3       99
4      873
5      696
6     1794
7       87
8     1542
9     1652
10    6733
11    4337
12    1162
13     360
14      66
dtype: int64
таблица сопряженности
             0      1  sum_rec
2.0          1     64       65
3.0          3     96       99
4.0         38    835      873
5.0         41    655      696
6.0        136   1658     1794
7.0         14     73       87
8.0        117   1425     1542
9.0        183   1469     1652
10.0       745   5988     6733
11.0       218   4119     4337
12.0        55   1107     1162
13.0        25    335      360
14.0         4     62       66
sum_rows  1580  17886    19466


## Вычисляем прогнозируемые значения

In [294]:
df3 = pandas.DataFrame(columns=range(k), index=numpy.array(all_groups.index))
summary = []
for i, row in df2.iterrows():
    for k in interval:
        df3.set_value(i, k, df2[k][i]**2/(df2[k]['sum_rows']*row['sum_rec']))
    summary.append(df3.ix[i].sum())
df3['sum_rec'] = summary
df3.loc['sum_rows'] = 0
list1 = [df3[i].sum() for i in interval]
for i in interval:
    df3.set_value('sum_rows', i, list1[i])
df3.set_value('sum_rows', 'sum_rec', df3['sum_rec'].sum())
#статистика хи квадрат
xi2 = (df3['sum_rec']['sum_rows']-1)*df2['sum_rec']['sum_rows']
#число степеней свободы
df = (len(interval)-1)*((len(df3.index)-1)-1)
#p-значение
p_val = scipy.stats.chisqprob(xi2, df)
print(xi2,p_val)

209.397103948 3.72869291968e-38


/Users/anastasiapanfilova/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:5: FutureWarning: scalar indexers for index type Index should be integers and not floating point


### Функция 

In [309]:
def filter_data(data, id_book_cell_name, id_score_cell_name, class_num, k):
    print("Ведем разбор для класса ",class_num)
    #выделяем подмассив с текущим классом
    filter_class = pandas.DataFrame(data[(data['ClassNumber'] == class_num) & (data[id_score_cell_name] == data[id_score_cell_name])],columns=[id_book_cell_name,id_score_cell_name])
    describe = filter_class[id_score_cell_name].describe()
    print(describe)
    #max
    print("min ", describe['min'])
    #min
    print("max ", describe['max'])
    divider = round((describe['max'] - describe['min'])/2, 1)
    print("divider ", divider)
    #общее количество записей по каждой книге
    all_groups = filter_class.groupby(id_book_cell_name).size()
    print(all_groups)
    # ключи книг
    #numpy.array(all_groups.index)
    # в dataframe по строкам обозначим учебники, а по столбцам интервалы
    df2 = pandas.DataFrame(columns=range(k), index=numpy.array(all_groups.index))
    interval = range(k)
    #print(df2)
    #идем по учебникам
    for i, row in df2.iterrows():
        first_divider,  second_divider = describe['min'], describe['min'] + divider
        #идем по интервалам
        for t in interval:
            if t == interval[len(interval)-1]:
                second_divider +=1
            #фильтруем оценки, которые укладываются в интервал
            count = filter_class[(filter_class[id_book_cell_name] == i) & (filter_class[id_score_cell_name] >= first_divider) & (filter_class[id_score_cell_name] < second_divider)]
            df2.set_value(i, t, len(count[id_book_cell_name]))
            #print(first_divider, second_divider)
            first_divider,  second_divider = first_divider + divider, second_divider + divider
    print("таблица сопряженности")
    df2['sum_rec'] = all_groups
    df2.loc['sum_rows'] = 0
    list1 = [df2[i].sum() for i in interval]
    for i in interval:
        df2.set_value('sum_rows', i, list1[i])
    df2.set_value('sum_rows', 'sum_rec', df2['sum_rec'].sum())
    #print(df2)
    return df2

In [310]:
def crosstabs(data, id_book_cell_name, id_score_cell_name, class_num, count_interval):
    df2 = filter_data(data, id_book_cell_name, id_score_cell_name, class_num, count_interval)
    print(df2)
    filter_class = pandas.DataFrame(data[(data['ClassNumber'] == class_num) & (data[id_score_cell_name] == data[id_score_cell_name])],columns=[id_book_cell_name,id_score_cell_name])
    #общее количество записей по каждой книге
    all_groups = filter_class.groupby(id_book_cell_name).size()
    interval = range(count_interval)
    df3 = pandas.DataFrame(columns=range(count_interval), index=numpy.array(all_groups.index))
    summary = []
    #вычисляем прогнозируемые значения по формуле n**2ij /(ni*n*j) 
    for i, row in df2.iterrows():
        for k in interval:
            df3.set_value(i, k, df2[k][i]**2/(df2[k]['sum_rows']*row['sum_rec']))
        summary.append(df3.ix[i].sum())
    df3['sum_rec'] = summary
    df3.loc['sum_rows'] = 0
    list1 = [df3[i].sum() for i in interval]
    for i in interval:
        df3.set_value('sum_rows', i, list1[i])
    df3.set_value('sum_rows', 'sum_rec', df3['sum_rec'].sum())
    print(df3)
    #статистика хи квадрат
    xi2 = (df3['sum_rec']['sum_rows']-1)*df2['sum_rec']['sum_rows']
    #число степеней свободы
    df = (len(interval)-1)*((len(df3.index)-1)-1)
    #p-значение
    p_val = scipy.stats.chisqprob(xi2, df)
    print("stat ",xi2, "chisqprob ", p_val)

In [320]:
crosstabs(data, "BooksMaID", "ScoreObyazMA", 1, 2)

Ведем разбор для класса  1
count    19760.000000
mean         8.715992
std          1.434328
min          0.000000
25%          8.000000
50%          9.000000
75%         10.000000
max         10.000000
Name: ScoreObyazMA, dtype: float64
min  0.0
max  10.0
divider  5.0
BooksMaID
1       29
3      128
4      712
5      734
6     2211
7       27
8      985
9     2023
10    7008
11    3848
12     894
13     243
14     671
dtype: int64
таблица сопряженности
            0      1  sum_rec
1.0         1     28       29
3.0         0    128      128
4.0        11    701      712
5.0         7    727      734
6.0        39   2172     2211
7.0         0     27       27
8.0        14    971      985
9.0        35   1988     2023
10.0      141   6867     7008
11.0       30   3818     3848
12.0       19    875      894
13.0        4    239      243
14.0        9    662      671
sum_rows  310  19203    19513
                    0           1   sum_rec
1.0       0.000111235  0.00140783  0.001519
3.0 

/Users/anastasiapanfilova/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:13: FutureWarning: scalar indexers for index type Index should be integers and not floating point


In [316]:
crosstabs(data, "BooksRuID", "ScoreObyazRU", 1, 2)

Ведем разбор для класса  1
count    19606.000000
mean        11.000255
std          3.320160
min          0.000000
25%          9.000000
50%         11.000000
75%         14.000000
max         16.000000
Name: ScoreObyazRU, dtype: float64
min  0.0
max  16.0
divider  8.0
BooksRuID
1     1186
2     2788
3       55
5     2502
6     1686
8      143
9       53
10    1216
11    6583
12     199
13    2207
14     369
16     378
dtype: int64
таблица сопряженности
             0      1  sum_rec
1.0        165   1021     1186
2.0        413   2375     2788
3.0         10     45       55
5.0        426   2076     2502
6.0        220   1466     1686
8.0         24    119      143
9.0          1     52       53
10.0       160   1056     1216
11.0      1056   5527     6583
12.0        18    181      199
13.0       349   1858     2207
14.0        43    326      369
16.0       103    275      378
sum_rows  2988  16377    19365
                    0           1   sum_rec
1.0         0.0076825   0.0536701

/Users/anastasiapanfilova/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:13: FutureWarning: scalar indexers for index type Index should be integers and not floating point


In [319]:
crosstabs(data, "TeacherRank", "ScoreObyazMA", 3, 2) 

Ведем разбор для класса  3
count    19466.000000
mean        11.459776
std          2.639316
min          0.000000
25%         10.000000
50%         12.000000
75%         13.000000
max         15.000000
Name: ScoreObyazMA, dtype: float64
min  0.0
max  15.0
divider  7.5
TeacherRank
Вторая категория                     2187
Высшая категория                     9958
Первая категория                     5695
Соответствие занимаемой должности    1626
dtype: int64
таблица сопряженности
                                      0      1  sum_rec
Вторая категория                    240   1947     2187
Высшая категория                    673   9285     9958
Первая категория                    508   5187     5695
Соответствие занимаемой должности   159   1467     1626
sum_rows                           1580  17886    19466
                                            0          1   sum_rec
Вторая категория                    0.0166693  0.0969103  0.113580
Высшая категория                    0.0287873